In [ ]:

import time, requests
from bs4 import BeautifulSoup

from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re,csv
import gspread

# Initialize variables to store data

name=''
address=''
phone=''
website=''

# Path to your ChromeDriver executable. Replace with your own path.
# chromedriver_path='<path_to_your_chromedriver>'


chromedriver_path=r'C:\rds\Python\chromedriver-win64\chromedriver-win64\chromedriver.exe'
driver = webdriver.Chrome(executable_path=chromedriver_path)





In [ ]:
# Function to Get user input for location and industry

def get_url():
    # location = "mumbai"
    # industry = "TIAA India Private Limited"
    # location = "nashik"
    # industry = "TDK India Private Limited"
    location = input("Enter Location: ")
    industry = input("Enter industry: ")

    print("Entered: = ",location,' Entered Industry: = ',industry)
    
    # Construct the URL for Google Maps search
    url = f"https://www.google.com/maps/search/{industry}+{location}"
    driver.get(url)

In [ ]:
# Function to extract data from a line based on a given word
# Check if a line starts with a specific word and return the rest of the line

def line_starts_with_word(line, word):
    pattern = re.compile(rf"^(?i){re.escape(word)}\s*:\s*(.*)")
    match = pattern.match(line)
    if match:
        return match.group(1).strip()  # Extract the rest of the line (excluding the word and colon)
    else:
        return None

In [ ]:
def get_all_data():
    global name, address, phone, website
    w1="name"
    w2="address"
    w3="phone"
    w4="website"

    # Get data from the webpage

    name = driver.find_element(By.CLASS_NAME,'DUwDvf').text
    all=driver.find_elements(By.CLASS_NAME,'CsEnBe')

    for a in all:
        # print(a.get_attribute('aria-label'))
        if line_starts_with_word(a.get_attribute('aria-label'),w2):
            # address=a.get_attribute('aria-label')
            address=line_starts_with_word(a.get_attribute('aria-label'),w2)
        elif line_starts_with_word(a.get_attribute('aria-label'),w3):
            # phone=a.get_attribute('aria-label')
            phone=line_starts_with_word(a.get_attribute('aria-label'),w3).replace(" ","")
    
        elif line_starts_with_word(a.get_attribute('aria-label'),w4):
            # phone=a.get_attribute('aria-label')
            website=line_starts_with_word(a.get_attribute('aria-label'),w4).replace(" ","")
    return address,name, phone, website

In [ ]:
# Function to store data in a CSV file
def store_data_in_csv(name, address, website, phone, filename='data.csv'):
    # Check if the file exists, if not create it and write the header
    columns = ['name', 'address', 'website', 'phone']
    try:
        with open(filename, 'x', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=columns)
            writer.writeheader()
    except FileExistsError:
        pass  # File already exists, no need to write header

    # Append the data to a new row
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=columns)
        writer.writerow({'name': name, 'address': address, 'website': website, 'phone': phone})


In [ ]:
# Function to append data to a Google Sheet

def google_sheet():
    # Replace 'creds.json' with your own Google Sheets API credentials file.
    # gc = gspread.service_account(filename='<path_to_your_credentials_file>')

    gc = gspread.service_account(filename='creds.json') # Replace with the actual filename

    sh=gc.open("<your_google_sheet_name>").sheet1    # Open the Google Sheet by its name. Replace with your own sheet's name.
    sh.append_row([name,address,phone])

In [ ]:
# pune, TIAA India Private Limited
# get_url pune, Siemens Digital Industries Software India Pvt Ltd

# Call the functions to perform the required actions

get_url()
get_all_data()
store_data_in_csv(name, address, website, phone)
google_sheet()
print("Success")

In [ ]:
# Some industries names 
# location ='pune'
# industries = ['Exacore IT Solutions Private Limited', 'Globant India Pvt. Ltd', 'Sasken Technologies Limited', 'DevLogic Technologies', 'Neosoft Technologies', 'Cisco Systems Pvt Ltd', 'Bentley Systems', 'Xento Systems', 'Red Hat India Private Limited', 'Capita India Pvt. Ltd', 'Avaya India Pvt. Ltd', 'SYSTEMS PLUS TRANSFORMATIONS LLP', 'eInfochips', 'Techila Global Services', 'Reflexis Systems India Private Limited', 'AppDirect India', 'SAP India', 'Capsilon Software Pvt. Ltd', 'Meganexus Ltd', 'Manipal Technologies Ltd', 'Nesecure Networks Pvt. Ltd', 'SoftwareGen Technologies', 'BNY Mellon International Ops India Ltd', 'Softdel', 'Caresoft Global Private Limited', 'Datafortune Software Solutions Pvt. Ltd', 'Mitz Technologies', 'Isobar', 'Edzeal Technologies']
